In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
import scipy 
import scipy.stats as ss
import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest
sys.path.append('../Script/')
import FM_States
import FM_selection
import TF
#from FM_States import *
import rpy2
from rpy2.robjects.packages import importr
from sklearn.neighbors import KNeighborsClassifier
import random

base = importr('base')
CC = importr('ConsensusClusterPlus')
pheatmap = importr('pheatmap')

ROOT_DIR = os.path.abspath("../")

In [ ]:
para = {
    'output_dir': ROOT_DIR+"/Sample_output/Example2/",
    'input_fmf_file_drug': ROOT_DIR +  "/Sample_output/Example1/matrix_factor_mcf7.csv",
    'input_fmf_file_sh':ROOT_DIR + "/Sample_output/Example2/matrix_factor_sh_mcf7.csv",
    'annotation_col':ROOT_DIR + "/Sample_output/Example1/annotation_col.csv",
    'selected_factor':ROOT_DIR +"/Sample_output/Example1/Features_diff_all.csv",
    'meta':ROOT_DIR + '/Sample_input/Example2/MetaData_sig_info_sh_MCF7_96h.csv',
    
}

In [ ]:
output_dir = para['output_dir']
try:
    os.makedirs(output_dir)
except OSError:
    print ("Info: Creation of the directory failed!\nThe direcory of %s may already exsit!" % output_dir)
else:
    print ("Successfully created the directory %s " % output_dir)
    

In [ ]:
matrix_factor_sh_MCF7 = pd.read_csv(para['input_fmf_file_sh'], index_col = 'Unnamed: 0')
matrix_factor = pd.read_csv(para['input_fmf_file_drug'], index_col = 'Unnamed: 0')
annotation_col_1 = pd.read_csv(para['annotation_col'], index_col= 'Unnamed: 0')
colors = ['red','pink','#fdd5ac','#8895df', 'cyan'] #Define the colors for cluster annotation
output_dir = para['output_dir']
select_modules =pd.read_csv(para['selected_factor'],index_col= 'Unnamed: 0')
meta_sh = pd.read_csv(para['meta'])  ## Annotation data for shRNA
meta_sh.index = meta_sh['sig_id']

In [ ]:
select_modules

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import random
y = annotation_col_1.loc[list(matrix_factor.index.values)]  ## use the states after drug treatment as labels;
x = matrix_factor[list(select_modules)].values              ## use the states after drug treatment as training set;
y_test = matrix_factor_sh_MCF7[list(select_modules)]        ## use the FM-factors after shRNA knockdown as the testing set;

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
#import random
#y = annotation_col_1.loc[list(matrix_factor.index.values)]  ## use the states after drug treatment as labels;
#x = matrix_factor.values              ## use the states after drug treatment as training set;
#y_test = matrix_factor_sh_MCF7       ## use the FM-factors after shRNA knockdown as the testing set;

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

#split dataset into train and test data
x_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=25223, stratify=y)

In [ ]:
dic_train = {}
dic_test = {}
score_dic_train = {}
score_dic_test = {}

score_cv = {}
score_cv_rf = {}
num_neighbors = -1
while num_neighbors < 30:
    num_neighbors = num_neighbors + 2
    score_list_train = []
    score_list_test = []
    
    
    neigh = KNeighborsClassifier(n_neighbors = num_neighbors)
    #neigh.fit(x_train, y_train) 
    rf_clf = RandomForestClassifier(max_depth = num_neighbors)
    
    cv_scores = cross_val_score(neigh, x, y['States'].values, cv=5, scoring = 'accuracy')
    cv_scores_rf = cross_val_score(rf_clf, x, y['States'].values, cv=5, scoring = 'accuracy')

    #print each cv score (accuracy) and average them
    print(cv_scores)
    ave_score = (np.mean(cv_scores))
    score_cv[num_neighbors] = [ave_score, np.std(cv_scores)]
    
    
    print(cv_scores_rf)
    ave_score_rf = (np.mean(cv_scores_rf))
    score_cv_rf[num_neighbors] = [ave_score_rf, np.std(cv_scores_rf)]
    
    #score_list_train.append(neigh.score(x_train,y_train))
    #score_list_test.append(neigh.score(X_val, y_val))
    
    #score_dic_train[num_neighbors] = score_list_train
    #score_dic_test[num_neighbors] = score_list_test
#score_df_train = pd.DataFrame.from_dict(score_dic_train)
#score_df_test = pd.DataFrame.from_dict(score_dic_test)
score_cv

In [ ]:
score_cv_rf

In [ ]:
score_cv

In [ ]:
k_list = []
mean_list = []
sd_list = []
uper_list = []
lower_list = []
for k in score_cv:
    k_list.append(k)
    mean_list.append(score_cv[k][0])
    uper_list.append(score_cv[k][0] + score_cv[k][1] )
    lower_list.append(score_cv[k][0] - score_cv[k][1] )
    #sd_list.append(score_cv[k][1])

In [ ]:
sd_list

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(4, 4))

#plt.plot(range(0,100), score_df_train, 'grey')

plt.plot(k_list, mean_list, 'black','o')
plt.plot(k_list, lower_list, 'grey', linestyle='dashed')
plt.plot(k_list, uper_list, 'grey', linestyle='dashed')
plt.axis([0, 30, 0.6, 0.8])

plt.xlabel("K")
plt.ylabel("Accuracy")
#plt.plot(range(0,12), score_list_test, 'red')
plt.tight_layout()
plt.savefig(output_dir+'/AUC_for_K.png', dpi=300)
plt.show()

In [ ]:
score_cv